In [2]:
#!pip install -U graphviz scikit-learn neptune-xgboost xgboost hyperopt mlflow deepchecks
#!pip install -U "neptune[xgboost]"

In [3]:
import pandas as pd
import numpy as np
# from google.colab import drive
import os

In [4]:
# drive.mount('/content/gdrive')

## **READING AND MERGING DATA**

In [7]:
# SPECIFY THE FILE PATH OF THE EXCEL FILE CONTAINING THE DATASET
path = r'C:\Users\hecto\Documents\PROJECTS\Python\Loan Default Prediction Using ML Models\Credit_Risk_Dataset.xlsx'

# CALL THE FUNCTION TO READ THE EXCEL DATA
sheet_names = ['loan_information', 'Employment', 'Personal_information', 'Other_information']

loan_information = pd.read_excel(path, 'loan_information')
employment = pd.read_excel(path, 'Employment')
personal_information = pd.read_excel(path, 'Personal_information')
other_information = pd.read_excel(path, 'Other_information')

In [8]:
# Merge 'loan information' and 'Employment' dataframes based on User_id
merged_df = pd.merge(loan_information, employment, left_on='User_id', right_on='User id')

# Merge the previously merged dataframe with 'personal information' based on 'User_id'
merged_df = pd.merge(merged_df, personal_information, left_on='User_id', right_on='User id')

# Merge the previously merged dataframe with 'other information' based on 'User_id'
merged_df = pd.merge(merged_df, other_information, left_on='User_id', right_on='User_id')

df = merged_df

# Display the first few rows of the merged dataframe
df.head()


,User_id,Loan Category,Amount,Interest Rate,Tenure(years),User id_x,Employmet type,Tier of Employment,Industry,Role,...,Home,Pincode,Social Profile,Is_verified,Delinq_2yrs,Total Payement,Received Principal,Interest Received,Number of loans,Defaulter
0,7013527,Consolidation,55884.0,11.84,6,7013527,Salaried,B,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGAZSEWdkcndwkcnCCM,...,rent,XX852X,No,NaN,0,1824.150000,971.46,852.69,0,1
1,7014291,Consolidation,55511.0,16.94,4,7014291,Self - Employeed,D,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGNCSEWdkcndwkcnCCM,...,mortgage,XX286X,NaN,Source Verified,0,22912.532998,18000.00,4912.53,0,0
2,7014327,Consolidation,12289.0,11.84,6,7014327,NaN,NaN,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGNYSEWdkcndwkcnCCM,...,own,XX113X,No,NaN,0,7800.440000,4489.76,3310.68,0,1
3,7014304,Credit Card,29324.0,14.71,4,7014304,NaN,NaN,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGCASEWdkcndwkcnCCM,...,rent,XX941X,Yes,NaN,0,6672.050000,5212.29,1459.76,0,0
4,7031995,Credit Card,30252.0,14.71,4,7031995,NaN,NaN,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGCASEWdkcndwkcnCCM,...,rent,XX913X,No,Verified,0,11793.001345,10000.00,1793.00,0,0


# **EXPLORATORY DATA ANALYSIS AND DATA PREPARATION**

**GOAL TO UNDERSTAND:**


*   **Identify data types of each column**
*   **Understand basic statistics of all numerical columns**
*   **Check Missing Values**
*   **Plan for Handling Missing Values**
--------------------------------------------------------------------------------
*   **Analyze the distribution of each column**
*   **Assess the quality of data based on the distribution**
*   **Identify any skewness in the data**
*   **Investigate the reasons behind data skewness**
--------------------------------------------------------------------------------
*   **Identify and handle categorical values**
*   **Identify and handle ordinal features**
--------------------------------------------------------------------------------
*   **Examine the correlation between different numeric variables**
*   **Examine the correlation between categorical variables**
--------------------------------------------------------------------------------
*   **Fix data imbalance**









In [9]:
# Display data types for each column in the dataframe. Goal is to see if there is any column with the wrong data type
df.dtypes

User_id                 int64
Loan Category          object
Amount                float64
Interest Rate         float64
Tenure(years)           int64
User id_x               int64
Employmet type         object
Tier of Employment     object
Industry               object
Role                   object
Work Experience        object
Total Income(PA)      float64
User id_y               int64
Gender                 object
Married                object
Dependents              int64
Home                   object
Pincode                object
Social Profile         object
Is_verified            object
Delinq_2yrs             int64
Total Payement        float64
Received Principal    float64
Interest Received     float64
Number of loans         int64
Defaulter               int64
dtype: object

In [10]:
# Use of describe pandas function to learn basic statistics of all numerical columns in our DataFrame
df.describe()

,User_id,Amount,Interest Rate,Tenure(years),User id_x,Total Income(PA),User id_y,Dependents,Delinq_2yrs,Total Payement,Received Principal,Interest Received,Number of loans,Defaulter
count,1.437270e+05,1.118030e+05,143727.000000,143727.000000,1.437270e+05,1.437270e+05,1.437270e+05,143727.000000,143727.000000,143727.000000,143727.000000,143727.000000,143727.000000,143727.000000
mean,3.733474e+07,1.376458e+05,12.038703,4.514503,3.733474e+07,7.242256e+04,3.733474e+07,1.997113,0.276601,10840.882025,8407.001538,2352.729746,0.005573,0.093712
std,3.182811e+07,1.575427e+05,3.880204,0.874241,3.182811e+07,5.539328e+04,3.182811e+07,1.411317,0.789019,8613.421536,7224.396032,2447.759754,0.098785,0.291429
min,2.080360e+05,0.000000e+00,5.420000,4.000000,2.080360e+05,4.000000e+03,2.080360e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.276118e+06,2.837950e+04,9.180000,4.000000,3.276118e+06,4.500000e+04,3.276118e+06,1.000000,0.000000,4674.775000,3104.160000,841.685000,0.000000,0.000000
50%,5.766517e+07,7.660300e+04,11.840000,4.000000,5.766517e+07,6.180000e+04,5.766517e+07,2.000000,0.000000,8212.410000,6000.000000,1632.240000,0.000000,0.000000
75%,6.729174e+07,2.054645e+05,14.440000,6.000000,6.729174e+07,8.600000e+04,6.729174e+07,3.000000,0.000000,14175.769677,11664.365000,2904.770000,0.000000,0.000000
max,7.895894e+07,8.000078e+06,23.540000,6.000000,7.895894e+07,7.141778e+06,7.895894e+07,4.000000,22.000000,57777.579870,35000.010000,24205.620000,5.000000,1.000000


In [11]:
# Check count of missing values in each column in the DataFrame
df.isnull().sum()

User_id                   0
Loan Category             0
Amount                31924
Interest Rate             0
Tenure(years)             0
User id_x                 0
Employmet type        84642
Tier of Employment    84642
Industry                  4
Role                      0
Work Experience           4
Total Income(PA)          0
User id_y                 0
Gender                    0
Married               48143
Dependents                0
Home                      0
Pincode                   0
Social Profile        47856
Is_verified           35803
Delinq_2yrs               0
Total Payement            0
Received Principal        0
Interest Received         0
Number of loans           0
Defaulter                 0
dtype: int64

# **ANALYSIS HANDLING MISSING VALUES**



*   Social Profile: Create a new category for NA values
*   Is verified: Create a new category for NA values
*   Married: Create a new category for NA values
*   Industry: Consider dropping missing values (4)
*   Work Experience: Consider dropping missing values (4)
*   Amount: Evaluate the impact of removing rows with missing values on the data distribution
*   Employment type: Determine the appropriate approach for handling missing values
*   Tier of Employment: Determine the appropriate approach for handling missing values



In [12]:
# Drop rows with missing values in the 'Industry' and 'Work Experience' columns as the data in both columns is meaniningless due to the small amount of rows with missing values
df = df.dropna(subset=['Industry', 'Work Experience'])

In [13]:
# Replace null values 'NA' with 'Missing' value in columns 'Social Profile', 'Is verified', 'Married', 'Employment type'
df.rename(columns={'Total Payement ': 'Total Payment', 'Employmet type':'Employment Type'}, inplace=True)
df.fillna({'Social Profile':'Missing', 'Is_verified':'Missing', 'Married':'Missing', 'Employment Type':'Missing'}, inplace=True)
df.head()

C:\Users\hecto\AppData\Local\Temp\ipykernel_15480\3742823313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Total Payement ': 'Total Payment', 'Employmet type':'Employment Type'}, inplace=True)
C:\Users\hecto\AppData\Local\Temp\ipykernel_15480\3742823313.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna({'Social Profile':'Missing', 'Is_verified':'Missing', 'Married':'Missing', 'Employment Type':'Missing'}, inplace=True)


,User_id,Loan Category,Amount,Interest Rate,Tenure(years),User id_x,Employment Type,Tier of Employment,Industry,Role,...,Home,Pincode,Social Profile,Is_verified,Delinq_2yrs,Total Payment,Received Principal,Interest Received,Number of loans,Defaulter
0,7013527,Consolidation,55884.0,11.84,6,7013527,Salaried,B,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGAZSEWdkcndwkcnCCM,...,rent,XX852X,No,Missing,0,1824.150000,971.46,852.69,0,1
1,7014291,Consolidation,55511.0,16.94,4,7014291,Self - Employeed,D,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGNCSEWdkcndwkcnCCM,...,mortgage,XX286X,Missing,Source Verified,0,22912.532998,18000.00,4912.53,0,0
2,7014327,Consolidation,12289.0,11.84,6,7014327,Missing,NaN,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGNYSEWdkcndwkcnCCM,...,own,XX113X,No,Missing,0,7800.440000,4489.76,3310.68,0,1
3,7014304,Credit Card,29324.0,14.71,4,7014304,Missing,NaN,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGCASEWdkcndwkcnCCM,...,rent,XX941X,Yes,Missing,0,6672.050000,5212.29,1459.76,0,0
4,7031995,Credit Card,30252.0,14.71,4,7031995,Missing,NaN,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,KHMbckjadbckIFGCASEWdkcndwkcnCCM,...,rent,XX913X,No,Verified,0,11793.001345,10000.00,1793.00,0,0


In [14]:
# Create a new variable "amount_missing" to indice if the "Amount" is missing or not. Assign 1 if 'Amount' is null, otherwise assign 0.
df['amount_missing'] = np.where(df['Amount'].isnull(), 1, 0)

C:\Users\hecto\AppData\Local\Temp\ipykernel_15480\3181412511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['amount_missing'] = np.where(df['Amount'].isnull(), 1, 0)


In [15]:
#We change the null values in 'Amount' column to 'missing' category.
df.fillna({'Amount':-1000}, inplace=True)

C:\Users\hecto\AppData\Local\Temp\ipykernel_15480\2880490832.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna({'Amount':-1000}, inplace=True)


In [16]:
#We change the null values in 'Tier of Employment' column to 'Z' category.
df.fillna({'Tier of Employment':'Z'}, inplace=True)

C:\Users\hecto\AppData\Local\Temp\ipykernel_15480\2361938063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna({'Tier of Employment':'Z'}, inplace=True)


In [17]:
# Check once again count of missing values in each column in the DataFrame after previous transformations
df.isnull().sum()

User_id               0
Loan Category         0
Amount                0
Interest Rate         0
Tenure(years)         0
User id_x             0
Employment Type       0
Tier of Employment    0
Industry              0
Role                  0
Work Experience       0
Total Income(PA)      0
User id_y             0
Gender                0
Married               0
Dependents            0
Home                  0
Pincode               0
Social Profile        0
Is_verified           0
Delinq_2yrs           0
Total Payment         0
Received Principal    0
Interest Received     0
Number of loans       0
Defaulter             0
amount_missing        0
dtype: int64

# **DROP CATEGORICAL COLUMN WITH TOO MANY CATEGORIES**

In [18]:
# Print the number of unique values in all columns
df.nunique()

User_id               133748
Loan Category              7
Amount                 86157
Interest Rate            137
Tenure(years)              2
User id_x             133748
Employment Type            3
Tier of Employment         8
Industry               12974
Role                      46
Work Experience            7
Total Income(PA)       11380
User id_y             133748
Gender                     3
Married                    3
Dependents                 5
Home                       5
Pincode                  844
Social Profile             3
Is_verified                4
Delinq_2yrs               22
Total Payment         104199
Received Principal     42021
Interest Received      93859
Number of loans            5
Defaulter                  2
amount_missing             2
dtype: int64

**Observations**
*   Some columns are ordinal and those categorical variables would need to be trated differently during categorical encoding
*   Address the challenge of categorical columns with a large number of categories. Determine which categories fall into this category and develop a strategy for handling them



In [19]:
# Dropping 'Industry' and User_id columns as it does not have any significant information
df = df.drop(columns=["Industry", 'User_id', 'User id_x', 'User id_y', 'Pincode', 'Role'])
df.nunique()

Loan Category              7
Amount                 86157
Interest Rate            137
Tenure(years)              2
Employment Type            3
Tier of Employment         8
Work Experience            7
Total Income(PA)       11380
Gender                     3
Married                    3
Dependents                 5
Home                       5
Social Profile             3
Is_verified                4
Delinq_2yrs               22
Total Payment         104199
Received Principal     42021
Interest Received      93859
Number of loans            5
Defaulter                  2
amount_missing             2
dtype: int64

# **MULTICOLLINEARITY**

To assess the relationships between variables in the input DataFrame, we compute the Spearman correlation matrix that tells us how much two variables are correlated.
<br>By examining the heatmap, we can identify potential concerns related to multicollinearity which occurs when two or more independent variables exhibit high correlation. Multicollinearity can impact the interpretation of the model and lead to overfitting. In such instances, it may be necessary to remove one of the correlated variables to mitigate this issues.

In [20]:
# Multicollinearity is the ocurrence of high intercorrelations among two or more independent variables in a multiple regression mode
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = df.corr()
plt.figure(figsize=(15,15), dpi=600)
x = sns.heatmap(corr_matrix, square=True, center=0, annot=True, linewidths=.5, cmap="RdBu_r", cbar_kws={'shrink':0.8}, vmin=-1, vmax=1)
x.set_xticklabels(x.get_xticklabels(), rotation=45, fontsize=8, ha='right')
plt.title("Correlation Matrix")
plt.show()

ValueError: could not convert string to float: 'Consolidation'

**Observation from the heatmap**
No two variables have high correlation with each other, so there is no issue of multicollinearity. It is safe to use all variables in machine learning model building

**Spearman Correlation Coefficient**
<br>Spearman works best if there are nonlinear relationships between different variables

--------------------------------------------------------------------------------

**We can confirm non-linear relationship between features by looking at pair-wise scatter plots below**
There are non-linear relationships between features, so our decision of going with Spearman was CORRECT.


**Scatter Plots to visualize correlations amongst variables**
The following code generates a scatter plot matrix also known as a pair plot of all numeric features in the input DataFrame using the seaborn library.
The diagonal of the plot matrix shows a histogram of each variable's distribution. This allows for visual inspectino of the pairwise relationships between variables, which canbe useful for identifying patterns, trends, correlations, or potential outliers in the data.

In [ ]:
# Plot interaction scatter plots using seaborn

sns.pairplot(df)

# **Understand Skewness**

In [ ]:
import plotly.express as px
import numpy as np

In [ ]:
# Display features distribution histograms
def Feature_Distributions_Histogram(df):
    # Generate histograms for each column
    for c in df.columns:
        fig = px.histogram(df, x=c, hover_data=df.columns)
        fig.show()
Feature_Distributions_Histogram(df)

**Skewness**

*   Skewness is a statistical measure that describes the asymmetry or lack of asymmetry in a distribution. It provides insights into the shape of the distribution and the relative positioning of the mean, median and mode.
*   A positive skewness value indicates a right-skewed distribution, where the tail is elongated towards the right A negative skewness value indicates a left-skewed distribution where the tail is elongated towards the left. A skewness value of 0 indicates a symmetric distribution.
*   By quantifying skewness we can gain a numerical understanding of the distrubtion's asymmetry and further analyze its implications in data analysis and modeling.

**How to quantify skewness**
*   Skewness = 3 * mean (mean - median) / standard deviation
--------------------------------------------------------------------------------
**Question to consider**
*   Should we remove outliers/extreme values first and then fix skewness or the other way around?
--------------------------------------------------------------------------------
**Decision**
*   to address the presence  of outliers in skewed data, we'll first correct the skewness
*   One approach mitigating the impact of outliers is by utilizing z-score. The z-score measures how many standard deviations an individual data point is away from the mean. By setting a threshold, such as 3 standard deviations, data points that exceed this threshold can be considered as outliers and subsequently removed from the dataset.

**Let us print skewness in each feature and use log transformation to fix skewness.**

**Note**

It is important no note that there are numerous features in the dataset with a value of 0. to address this issue and normalize the data, we can apply a log transformation specifically to the non-zero values. By taking the logarithm of these values, we can achieve a more symmetric distribution and reduce the impact of extreme values. This transformation can be particularly useful when working with skewed data or variables that exhibit a wide range of values.

Let's only transform features if skewness is in the following range
*   **Skewness <-3 OR Skewness > 3**









In [ ]:
# Add all the features to check and fix skewness in features_log array
features_log = ['Amount','Interest Rate','Tenure(years)','Dependents','Total Payment','Received Principal','Interest Received']
def fix_skewness(df, features):
  features_log = features
  for f in features_log:
    sk=df[f].skew()
    print("Initial skewness in feature: ",f," is:", sk)

    if sk>3 or sk<-3:

      Log_Fare = df[f].map(lambda i: np.log(i) if i>0 else 0)
      df[f]= Log_Fare
      print("Final skewness in feature: ",f,"is: ", Log_Fare.skew())
      fig = px.histogram(Log_Fare, x=f)
      fig.show()
  return df
df = fix_skewness(df, features_log)

## **One-Hot Encoding of Categorical Features and Ordinal Encoding of Ordinal Features**

**Categorical and Ordinal Variables**
<br>Categorical variables refer to variables that represent discrete categories or labels, such as gender (male/female), or product types (A/B/C). These variables do not have a specific numerical order or hierarchy.
<br>On the other hand, ordinal variables also represent discrete categories, but they haven an inherent order or raking associated to them. Examples of ordinal variables include educational level (elementary/middle/high-school/college), employment status (unemployed/part-time/full-time), or customer satisfaction rating (low/medium/high).

### **Categorical Hot-Encoding**

In [ ]:
# Add all categorical features for categorical one-hot encoding in categorical_features array
data = df
categorical_features = ['Gender','Married','Home','Social Profile','Loan Category','Employment Type','Is_verified']

# Perform one-hot encoding using pandas get_dummies() function
encoded_data = pd.get_dummies(data, columns=categorical_features)

In [ ]:
encoded_data.head()

### **Ordinal Encoding**

In [ ]:
# Define the ordinal features array
ordinal_features=['Tier of Employment', 'Work Experience']

# Define the pandas DataFrame for encoding
data = encoded_data

# Create a custom mapping of categories to numerical  labels
tier_employment_order = list(encoded_data["Tier of Employment"].unique())
tier_employment_order.sort()
work_experience_order = [0,'<1', '1-2','2-3','3-5','5-10','10+']

custom_mapping = [tier_employment_order, work_experience_order]

# Create the function to perform ordinal encoding
# Call the corresponding library and module to import OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder
def perform_ordinal_encoding(data, ordinal_features, custom_mapping):

  # Perform ordinal encoding using OrdinalEncoder with the custom mapping
  encoder = OrdinalEncoder(categories=custom_mapping)
  data[ordinal_features]=encoder.fit_transform(data[ordinal_features])

  # Return the encoded DataFrame
  return data

# Call the function to perform ordinal encoding
data = perform_ordinal_encoding(data, ordinal_features, custom_mapping)

In [ ]:
data.head()

## **Fix Data Imbalance in the Target Variable**

### **Over-Sampling**
Increase the number of instances in the minory class (defaulters) by duplicating existing examples or generating synthetic examples to achieve a balanced dataset.

In [ ]:
data['Defaulter'].value_counts()

It is not recommended to use SMOTE method or technique when minority class is less than 10%.
<br>To solve imbalance on dependent variable, we will not proceed with SMOTE method due to minority class being equal to 10.34% of the mayority class, quite close to the limit stated by the above-mentioned recommendation.